# __키워드로 이미지 검색하기__

- 튜토리얼 난이도: ★☆☆☆☆
- 읽는데 걸리는 시간: 10분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_search/search_image_by_keyword.ipynb
- 참고 문서: [음식 이미지 및 영양정보 텍스트 소개 데이터 세트](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=74)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">키워드-이미지 검색 서비스 이해하기</h4>
    <p>ThanoSQL에서는 키워드를 통해 이미지를 결과로 돌려받을 수 있는 검색 기능을 제공합니다. 이미지 분류 모델 등을 이용해서 사용자가 원하는 키워드를 목푯값(Target)으로 설정하고, 학습 된 모델을 사용해서 업데이트 되는 이미지를 색인한 컬럼을 추가합니다. 텍스트-이미지 검색 기능이 이미지로부터 의미를 분석하고 유사한 이미지를 제공한다면 키워드-이미지 검색은 원하는 목푯값(범주)에 해당하는 이미지를 찾아줍니다.</p>
</div>

검색은 "책이나 컴퓨터에서 목적에 따라 필요한 자료들을 찾아내는 일"이라는 사전적인 의미를 가지고 있습니다. ThanoSQL 키워드-이미지 검색은 특정 단어(키워드)의 포함 여부로 데이터베이스 내의 정보를 검색하는 방식과는 조금 다르게 작동합니다. 키워드 기반 이미지 검색은 이미지의 특징으로부터 단어를 미리 학습하고 예측하는 모델을 만들고, 해당 모델을 통해 특정 키워드에 포함 될 확률이 가장 높은 이미지를 제공합니다.

__아래는 ThanoSQL 키워드 이미지 검색 알고리즘의 활용 및 예시 입니다.__

- 쇼핑 카테고리의 범주 등을 학습 데이터로 활용하여 학습모델을 만들고 이를 이용해서 기존/신규 이미지에 색인 컬럼을 만듭니다. 해당 색인 컬럼은 이미지 등록일자 등과 같은 수치형 특성들과 결합해서 더욱 정교한 검색을 제공합니다.
- 이어지는 튜토리얼에서 다루는 유사 이미지 검색 결과나 텍스트-이미지 검색 결과 등 다양한 결과를 같이 활용해서 나만의 이미지 검색 서비스를 만들 수 있습니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 ThanoSQL에서 제공하는 비정형 데이터 검색 구문인 "<strong>SEARCH</strong>" 쿼리 구문과 기존 PostgreSQL에서 제공하는 정형 데이터 검색 구문인 "<strong>SELECT</strong>" 쿼리 구문을 같이 사용하여 특정 키워드를 이용해 원하는 이미지 검색을 진행합니다.</p>
</div>

<div class="admonition tip">
    <h4 class="admonition-title">데이터 세트 설명</h4>
    <p>음식 이미지 및 영양정보 텍스트 소개 데이터 세트는 과학기술정보통신부가 주관하고 한국지능정보사회진흥원이 지원하는 '인공지능 학습용 데이터 구축사업'으로 구축된 데이터로 한국인 다빈도 섭취 외식 메뉴와 한식메뉴 400종을 선정하여 양질의 이미지 데이터로 구성이 되어 있습니다. 842,000 장의 이미지로 구성되어 있으며, 본 튜토리얼에서는 해당 데이터 세트에서 일부(10종, 1,190장)의 사진만을 사용합니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET diet_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY diet
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/diet_data/diet.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

키워드-이미지 검색 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __diet__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [4]:
%%thanosql
SELECT *
FROM diet
LIMIT 5

,image_path,label
0,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과
1,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과
2,thanosql-dataset/diet_data/diet/백향과/1_A220148X...,백향과
3,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과
4,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>diet</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>image_path: 이미지 경로</li>
        <li>label: 이미지 라벨</li>
    </ul>
</div>

## __2. 키워드 검색 모델 생성__

이전 단계에서 확인한 __diet__ 테이블을 사용하여 이미지 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>diet_image_classification</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 3 min)

In [5]:
%%thanosql
BUILD MODEL diet_image_classification
USING ConvNeXt_Tiny
OPTIONS (
    image_col='image_path',
    label_col='label',
    max_epochs=1,
    overwrite=True
    )
AS
SELECT *
FROM diet

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>diet_image_classification</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>ConvNeXt_Tiny</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"image_col": 데이터 테이블에서 이미지의 경로를 담은 컬럼 (str, default: 'image_path')</li>
            <li>"label_col": 목푯값의 정보를 담은 컬럼의 이름 (str, default: 'label')</li>
            <li>"max_epochs": 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, optional, default: 3)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 생성된 모델을 사용하여 키워드-이미지 검색__

이번 단계에서는 __diet_image_classification__ 모델과 테이블을 사용하여 특정 이미지의 목푯값을 예측합니다.

In [6]:
%%thanosql
PREDICT USING diet_image_classification
OPTIONS (
    image_col='image_path',
    result_col='predict_result'
    )
AS 
SELECT *
FROM diet

,image_path,label,predict_result
0,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과,백향과
1,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과,백향과
2,thanosql-dataset/diet_data/diet/백향과/1_A220148X...,백향과,백향과
3,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과,백향과
4,thanosql-dataset/diet_data/diet/백향과/0_A220148X...,백향과,백향과
...,...,...,...
1185,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
1186,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
1187,thanosql-dataset/diet_data/diet/사과파이/1_A020511...,사과파이,사과파이
1188,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>diet_image_classification</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>"image_col": 예측에 사용할 이미지의 경로가 기록되어 있는 컬럼의 이름 (str, default: 'image_path')</li>
            <li>"result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

이제 "__PREDICT USING__", "__SELECT__", "__WHERE__" 쿼리 구문을 사용하여 특정 조건의 데이터만을 검색합니다. 다음 쿼리 구문을 실행하여 label이 '사과파이'이고, 예측 결과 또한 '사과파이'인 데이터만을 검색합니다.

In [7]:
%%thanosql
SELECT *
FROM (
    PREDICT USING diet_image_classification
    AS
    SELECT *
    FROM diet
    )
WHERE label=predict_result
AND label LIKE '사과파이'
LIMIT 10

,image_path,label,predict_result
0,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
1,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
2,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
3,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
4,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
5,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
6,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
7,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
8,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이
9,thanosql-dataset/diet_data/diet/사과파이/0_A020511...,사과파이,사과파이


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SELECT * FROM (...)</strong>" 쿼리 구문을 통해 "<strong>PREDICT USING</strong>"으로 시작하는 쿼리 구문의 결과를 모두 선택합니다.</li>
        <li>"<strong>WHERE</strong>" 쿼리 구문을 통해 조건을 설정합니다. 이 조건은 "<strong>AND</strong>"를 통해 이어집니다.
        <ul>
            <li>"label=predict_result": label 컬럼과 predict_result 컬럼의 값이 같은 데이터만 추출합니다.</li>
            <li>"label LIKE '사과파이'": label 컬럼이 '사과파이'인 데이터만 추출합니다.</li>
        </ul>
        </li>
    </ul>
</div>

## __5. 튜토리얼을 마치며__

이번 튜토리얼에서는 음식 이미지 데이터 세트를 사용하여 키워드와 관련된 음식 이미지를 검색하는 모델을 만들어 보았습니다. 이번 튜토리얼에서는 정확도 보다는 작동위주로 빠르게 진행되었습니다. 빌드 옵션의 학습 횟수, 데이터 수등 다양한 옵션들을 조절하여 모델의 정확도를 향상시킬 수 있습니다. 이어지는 유사이미지 검색모델, 텍스트 설명을 이용한 이미지 검색 튜토리얼을 진행하고 나만의 다양한 검색 서비스를 만들어 보세요.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>키워드-이미지 검색 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>